**Chapter 2 – End-to-end Machine Learning project**

# Contents
1. Get the data
2. Data analysis
3. Data preparation
4. Select, train and fine-tune a model
5. Evaluate the models on the test set


Your task is to predict median house values in Californian districts, given a number of features from these districts. A district typically has a population of 600 to 3000 people.

This is a regression task, since you are asked to predict a numerical value.

## Setup

In [ ]:
# Common imports
import sklearn
import numpy as np

# Import the SSL module and configure certificate verification to be optional to avoid SSLCertVerificationError.
import ssl
ssl.SSLContext.verify_mode = ssl.VerifyMode.CERT_OPTIONAL

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Get the data

In [ ]:
# This function creates a dataset/housing directory subdirectory, if it doesn't already exists. Then it downloads 
# the housing.tgz file, and extracts the housing.csv file.

from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()

In [ ]:
# Display the top five rows
housing.head()

In [ ]:
# Get a quick description of the data
# (look for null values and non-numerical data which require special data preparation)
housing.info()

In [ ]:
# Look at the non-numerical feature, ocean_proximity
housing["ocean_proximity"].value_counts()

In [ ]:
# Display basic statistics for the features
housing.describe()

In [ ]:
# Plot a histogram of each numerical feature
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

### Observations in the histograms
1) housing_median_age, median_house_value and median_income have been capped. If this is a problem, we could
   remove the largest values (we will omit that here for simplicity)
2) median_income has been scaled to tens of thousand dollars.
3) Some histograms are tail-heavy (may be a problem for some algorithms). Although a feature can be made less
   tail-heavy by using simple techniques, such as replacing each value with its square root or the logarithm,
   we will omit that here for simplicity.
4) The features has different scales (scaling may be required)

# Data analysis

In [ ]:
# Perform a geographical scatterplot of the data
housing.plot(kind="scatter", x="longitude", y="latitude", grid=True, alpha=0.2)
plt.show()

In [ ]:
# A better visualization. The radius of each circle (option s) represents the district's population, and the
# color (option c) represents the price.
housing.plot(kind="scatter", x="longitude", y="latitude",
    s=housing["population"]/100, label="population",
    c="median_house_value", cmap="jet", colorbar=True,
    legend=True, sharex=False, figsize=(10,7))
plt.show()

### Correlations
The plot above shows that the prices are much related to the location and the population density.

Let us now compute and illustrate linear correlations between selected attributes.

In [ ]:
# How much does each feature correlate with the median house value?
# The correlation coefficient ranges from -1 (100% negative correlation) to 1 (100% positive correlation)
corr_matrix = housing.corr(numeric_only=True)
corr_matrix["median_house_value"].sort_values(ascending=False)

### Attribute combinations
The total number of rooms in a district is not very useful. Instead, we want the number of rooms per household. Similarly, we want the number of bedrooms per room instead of the total number of bedrooms in a district, and the population per household instead of the total population in a district.

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
# How much does each of the new attributes correlate with the median house value?
corr_matrix = housing.corr(numeric_only=True)
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# We will keep the new attributes and delete the original ones.
housing = housing.drop(columns="total_rooms")
housing = housing.drop(columns="total_bedrooms")
housing = housing.drop(columns="population")

# Data preparation

<ol>
<li>how to split the dataset in a training set and a test set</li>
<li>how to clean the data so that there are no missing values</li>
<li>how to re-scale attribute values</li>
<li>and how to handle a non-numerical feature</li>
</ol>

## Split the dataset

In [ ]:
# Split the dataset randomly in training set (80%) and test set (20%). Use a fixed random seed (42).
# As a rule of thumb, pick 20% for the test set, unless the dataset is very large.
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

### stratified sampling
Random sampling is fine, if the dataset is large enough. If not, stratified sampling should be considered. Stratified sampling ensures that the test set is representative of the whole dataset. The dataset is first divided into subgroups called strata (e.g. a stratum could represent an income group). Then the right number of instances for both training and test sets are picked from each stratum. Stratified sampling reduces sampling bias.

In [ ]:
# Remove the labels from the training set (the method returns a new set, and it does not affect the original one).
housing_predictors = train_set.drop(columns="median_house_value")
# Keep the labels in a separate set.
housing_labels = train_set["median_house_value"].copy()

## Data cleaning

Strategies:
<ol>
<li>Remove all rows with missing values. Use the DataFrame's dropna() method.</li>
<li>Remove all columns with missing values.</li>
<li>Replace missing values with some default value (mean, median, most frequent or some fixed value). Use Scikit-Learn's SimpleImputer class.</li>
</ol>

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

In [ ]:
# Remove the text attribute because median can only be calculated on numerical attributes.
housing_num = housing_predictors.select_dtypes(include=[np.number])

In [ ]:
# Compute the median of each attribute.
imputer.fit(housing_num)

In [ ]:
# The result is stored in the statistics_ instance variable.
imputer.statistics_

In [ ]:
# Replace missing values with the medians.
imputer.transform(housing_num)

In [ ]:
# Scikit-Learn transformers returns NumPy arrays even when they get a Pandas DataFrame as input.
housing_num_numpy = imputer.transform(housing_num)

# If you want, you can wrap the returned NumPy array in a DataFrame, with the following code:
housing_dataframe = pd.DataFrame(housing_num_numpy, columns=housing_num.columns, index=housing_num.index)
housing_dataframe.head()

In [ ]:
# You can also convert a Pandas DataFrame to a numpy array with the following code:
housing_dataframe.to_numpy()

## Feature scaling
Most ML algorithms don't perform well, when the numerical input attributes have different scales. They tend to focus more on features with a large range of values, and less on features with a small range of values. There are two common  ways of scaling:

Min-max scaling (also called normalization): values are rescaled so that they end up ranging from 0 to 1.
Standardization: values are rescaled so that they have zero mean and unit variance.

Standardization is much less affected by outliers, but the lack of a fixed range (0 to 1) is a problem for some algorithms (e.g. neural networks).

Scaling the target values is generally not required.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# scaler = StandardScaler()
scaler = MinMaxScaler()
housing_num_scaled = scaler.fit_transform(housing_num)
housing_num_scaled

## Handling non-numerical features

In [ ]:
# Extract the single non-numerical attribute, which is categorial.
housing_cat = housing_predictors[["ocean_proximity"]]

# Show the different categories.
housing_cat.value_counts()

In [ ]:
# Create one binary attribute per category.
# It is called one-hot encoding, because only one attribute value will be 1 (hot) while the others will be 0 (cold).
# The default output is a sparse matrix, which only stores the location of the non-zero elements. Here we will use
# a dense matrix (sparse=False), since there are only a few categories.
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(sparse_output=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

# Other ways of converting a categorical attribute:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

In [ ]:
# The original categories are stored in the categories_ instance variable.
cat_encoder.categories_

## Transformation pipelines
The Scikit-Learn Pipeline class can perform a sequence of transformations.

In [ ]:
# Pipeline for the numerical attributes.
from sklearn.pipeline import make_pipeline

num_pipeline = make_pipeline(SimpleImputer(strategy="median"), MinMaxScaler())

housing_num_transformed = num_pipeline.fit_transform(housing_num)

In [ ]:
# Pipeline for the categorical attribute.
cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(sparse_output=False))

In [ ]:
# Pipeline that will transform both the numerical and categorial attributes and combine them.
from sklearn.compose import ColumnTransformer

# We must pass the names of the attributes which should be transformed
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

preprocessing_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

# Alternative construct:
#from sklearn.compose import make_column_selector, make_column_transformer

#preprocessing_pipeline = make_column_transformer(
#    (num_pipeline, make_column_selector(dtype_include=np.number)),
#    (cat_pipeline, make_column_selector(dtype_include=object)),
#)

housing_predictors_prepared = preprocessing_pipeline.fit_transform(housing_predictors)

In [ ]:
housing_predictors_prepared

In [ ]:
# This method outputs the number of rows and columns in the dataset.
housing_predictors_prepared.shape

# Select, train and fine-tune a model

In [ ]:
# Create a DecisionTreeRegressor and add it to the preprocessing pipeline.
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

tree_reg = Pipeline([
    ("preprocessing", preprocessing_pipeline),
    ("decision_tree", DecisionTreeRegressor(random_state=42))
])

In [ ]:
# Use GridSearchCV to fine-tune hyperparameters for a DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

params = {'decision_tree__max_depth': list(range(10, 30)),'decision_tree__max_leaf_nodes': [400, 500, 600], 
          'decision_tree__min_samples_split': [40, 60, 80]}

grid_search = GridSearchCV(tree_reg, params, n_jobs=-1, cv=5)

# Train (beware that the preprocessing pipeline will run before each decision tree is trained)
grid_search.fit(housing_predictors, housing_labels)

In [ ]:
# Display parameters for the best estimator
grid_search.best_params_

In [ ]:
# Measure the models RMSE on the training set
from sklearn.metrics import mean_squared_error

# We input the unprepared predictors of the training set to the predict method, because the predict method
# will run the preprocessing_pipeline on the input data, before making predictions.
housing_predictions = grid_search.predict(housing_predictors)
grid_search_rmse = mean_squared_error(housing_labels, housing_predictions, squared=False)
grid_search_rmse

# Evaluate the model on the test set

In [ ]:
# Remove the labels from the test set.
X_test = test_set.drop("median_house_value", axis=1)
# Keep the labels in a separate set.
y_test = test_set["median_house_value"].copy()

In [ ]:
# Evaluate the DecisionTreeRegressor.
# Remember that the predict method will run the preprocessing_pipeline on X_test, before making predictions.
grid_search_predictions = grid_search.predict(X_test)
grid_search_rmse = mean_squared_error(y_test, grid_search_predictions, squared=False)
grid_search_rmse